In [32]:
import sys
sys.path.append("../src")
from mistral_client import run_mistral
from ner_post_processing import parse_entities_promptner, get_token_labels

import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from collections import defaultdict

In [33]:
dataset = load_dataset("DFKI-SLT/cross_ner", "politics")

class_labels = dataset["validation"].features["ner_tags"].feature.names + ["B-frenchpolitician", "I-frenchpolitician", "B-frenchpoliticalparty", "I-frenchpoliticalparty"]
index2label = {i: label for i, label in enumerate(class_labels)}
label2index = {v: k for k, v in index2label.items()}

label2index

{'O': 0,
 'B-academicjournal': 1,
 'I-academicjournal': 2,
 'B-album': 3,
 'I-album': 4,
 'B-algorithm': 5,
 'I-algorithm': 6,
 'B-astronomicalobject': 7,
 'I-astronomicalobject': 8,
 'B-award': 9,
 'I-award': 10,
 'B-band': 11,
 'I-band': 12,
 'B-book': 13,
 'I-book': 14,
 'B-chemicalcompound': 15,
 'I-chemicalcompound': 16,
 'B-chemicalelement': 17,
 'I-chemicalelement': 18,
 'B-conference': 19,
 'I-conference': 20,
 'B-country': 21,
 'I-country': 22,
 'B-discipline': 23,
 'I-discipline': 24,
 'B-election': 25,
 'I-election': 26,
 'B-enzyme': 27,
 'I-enzyme': 28,
 'B-event': 29,
 'I-event': 30,
 'B-field': 31,
 'I-field': 32,
 'B-literarygenre': 33,
 'I-literarygenre': 34,
 'B-location': 35,
 'I-location': 36,
 'B-magazine': 37,
 'I-magazine': 38,
 'B-metrics': 39,
 'I-metrics': 40,
 'B-misc': 41,
 'I-misc': 42,
 'B-musicalartist': 43,
 'I-musicalartist': 44,
 'B-musicalinstrument': 45,
 'I-musicalinstrument': 46,
 'B-musicgenre': 47,
 'I-musicgenre': 48,
 'B-organisation': 49,
 'I-o

In [34]:
fp = "../data/unlabeled/politics_tasklevel.txt"
fp_scored = "../data/scored/politics_tasklevel_french_kw.txt"

In [35]:
sentences = []
with open(fp, "r") as file:

        while True:
            line = file.readline()  # Read a single line
            if not line:  # Break out of the loop when the end of the file is reached
                break
            line = line.strip()
            sentences.append(line)

In [36]:
len(sentences)

501885

In [37]:
keywords = ["france", "french"]

sentences_french = [sentence for sentence in sentences if any(keyword in sentence.lower() for keyword in keywords)]

In [38]:
years = list(range(2012, 2025))
sentences_french_modern = [sentence for sentence in sentences_french if any(str(year) in sentence.lower() for year in years)]

In [39]:
sentences_french_modern

['On 22 July 2014 , King Philippe nominated Charles Michel ( MR ) and Kris Peeters ( CD & amp ; V ) to lead the formation of a new federal cabinet composed of the Flemish parties N-VA , CD & amp ; V , Open Vld and the French-speaking MR , which resulted in the Michel Government .',
 "More recently , the memory of Charles Martel has been appropriated by far right and white nationalist groups , such as the ' Charles Martel Group ' in France , and by Australia -born Brenton Harrison Tarrant , the alleged perpetrator of the Christchurch mosque shootings at Al Noor Mosque and Linwood Islamic Centre in Christchurch , New Zealand in 2019 .",
 'In early 2013 , documents provided by Edward Snowden were published by " The Washington Post " and " The Guardian " , 31 Oct 2013 , Lorenzo Franceschi-Bicchierai , mashable.com',
 'Before this , three territories of member states had left the EU or its forerunners , these being French Algeria ( in 1962 , upon independence ) , Greenland ( in 1985 , follo

In [40]:
len(sentences_french_modern)

656

In [41]:
prompt_french = lambda text: f"""
Dfn: An entity is a person (person), organisation (organisation), french politician (politician), french political party (politicalparty), event (event), election (election), 
country (country), location (location) or other political entity (misc). Dates, times, abstract concepts, adjectives, and verbs are not entities.

Example 1: In the 2014 European Parliament election in France , the National Front won the elections with 24.85 % of the vote , a swing of 18.55 % , winning 24 seats , up from 3 previously .

Answer:
1. 2014 European Parliament election | True | as it is an election (election)
2. France | True | as it is a country (country)
3. National Front | True | as it is a political party (frenchpoliticalparty)

Example 2: The FN received 33.9 % of the votes in the 2017 French presidential election , making it the largest Eurosceptic party in France .

Answer:
1. FN | True | as it is a political party (frenchpoliticalparty)
2. 2017 French presidential election | True | as it is an election (election)
3. Eurosceptic party | True | as it is a political party (frenchpoliticalparty)
4. France | True | as it is a country (country)

Example 3: The 2017 French presidential election caused a radical shift in French politics , as the prevailing parties of The Republicans and Socialists failed to make it to the second round of voting , with far-right Marine Le Pen and political newcomer Emmanuel Macron instead facing each other .

Answer:
1. 2017 French presidential election | True | as it is an election (election)
2. French politics | False | as it is an abstract concept, not a specific entity
3. The Republicans | True | as it is a political party (frenchpoliticalparty)
4. Socialists | True | as it is a political party (frenchpoliticalparty)
5. far-right | False | as it is an adjective describing a political orientation, not a specific entity
6. Marine Le Pen | True | as she is a French politician (frenchpolitician)
7. Emmanuel Macron | True | as he is a French politician (frenchpolitician)
8. political newcomer | False | as it is an abstract concept, not a specific entity

Q. Given the paragraph below, identify a list of possible entities and for each entry explain why it either is or is not an entity.

Paragraph: {text}
"""

In [42]:
scored = defaultdict(list)

for idx, text in enumerate(tqdm(sentences_french_modern)):
    if (idx + 1) % 100 == 0:
        df_scored = pd.DataFrame(scored)
        df_scored.to_csv(fp_scored, index=False)
    
    try:
        tokens = text.split()
        prompt_input = prompt_french(text)
        output = run_mistral(prompt_input)
        ner_tags = get_token_labels(text, parse_entities_promptner(output), label2index)

        scored["tokens"].append(text)
        scored["prompt"].append(prompt_input)
        scored["output"].append(output)
        scored["ner_tags"].append(ner_tags)
    except Exception as e:
        print(e)
        continue

df_scored = pd.DataFrame(scored)
df_scored.to_csv(fp_scored, index=False)

  0%|          | 1/656 [00:07<1:23:20,  7.63s/it]

'B-frenchpolitician'


  0%|          | 2/656 [00:19<1:51:42, 10.25s/it]

'B-frenchpolitician'


  0%|          | 3/656 [00:23<1:20:53,  7.43s/it]

...

'B-frenchpoliticalparty'


  3%|▎         | 18/656 [02:15<1:16:12,  7.17s/it]

'B-frenchpoliticalparty'


  3%|▎         | 18/656 [02:17<1:21:15,  7.64s/it]


KeyboardInterrupt: 